In [2]:
from langchain.document_loaders import PDFMinerLoader

loader = PDFMinerLoader("ug_cr_rptstd.pdf")
documents = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

texts[0]

Document(page_content='IBM Cognos Analytics - Reporting\nバージョン 11.1\n\nユーザー・ガイド\n\nIBM\n\n\x0c©\n\n製品情報\n\nこのドキュメントは IBM Cognos Analytics バージョン 11.1.0 を対象として作成されています。また、その後のリリースも\n対象となる場合があります。\n\n著作権\n\nLicensed Materials - Property of IBM\n\n© Copyright IBM Corp. 2005, 2021.\n\nUS Government Users Restricted Rights – Use, duplication or disclosure restricted by GSA ADP Schedule Contract with\nIBM Corp.\n\nIBM、IBM ロゴおよび ibm.com は、世界の多くの国で登録された International Business Machines Corporation の商標で\nす。 他の製品名およびサービス名等は、それぞれ IBM または各社の商標である場合があります。現時点での IBM の商標\nリストについては、www.ibm.com/legal/copytrade.shtml をご覧ください。\n\n以下は、各社の商標または登録商標です。\n\n• Adobe、Adobe ロゴ、PostScript、PostScript ロゴは、Adobe Systems Incorporated の米国およびその他の国における\n\n登録商標または商標です。', metadata={'source': 'ug_cr_rptstd.pdf'})

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_KEY

In [7]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
            api_key=OPENAI_API_KEY,
            model="text-embedding-ada-002"
)

from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings, persist_directory="./storage/openai")
db.persist()

In [8]:
retriever = db.as_retriever()
retriever.get_relevant_documents(query="クロス集計とは")

[Document(page_content='第 5 章クロス集計................................................................................................. 69\nクロス集計ノードとクロス集計ノードのメンバー................................................................................... 69\nクロス集計のプロパティーの設定.............................................................................................................70\n単一エッジのクロス集計の作成................................................................................................................ 72\nネストされたクロス集計の作成................................................................................................................ 72', metadata={'source': 'ug_cr_rptstd.pdf'}),
 Document(page_content='リストのクロス集計への変更.................................................................................................................... 76\nTM1 データ・ソースからのテキスト数値データのクロス集計への追加................................................. 76', metadata={'source': 'ug_cr_rptstd.pdf'}),
 Document(page_content='単純な集計の追加.

In [17]:
from langchain.vectorstores import Chroma
db = Chroma(persist_directory="./storage/openai/", embedding_function=embeddings)
retriever = db.as_retriever()

from langchain.llms import OpenAI
llm = OpenAI(temperature=0.7, max_tokens=1000)

from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

query = "生成されたSQLを見るにはどうすればよいですか？"
result = qa({"query": query})

In [18]:
result

{'query': '生成されたSQLを見るにはどうすればよいですか？',
 'result': ' ナビゲーション・メニューから「クエリー」フォルダーをクリックし、「その他」をクリックして「生成されたSQL/MDXを表示」を選択します。「生成されたSQL」ウィンドウが開き、レポートのクエリーが表示されます。「クエリー結果」ウィンドウでクエリー結果をクリックすると、クエリーのSQLが表示されます。',
 'source_documents': [Document(page_content='レポートの SQL の表示\nSQL を表示すると、レポートの実行時にデータベースに渡される情報を確認できます。\n\n始める前に\nレポートで編集モードにする必要があります。\n\n手順\n\n1. ナビゲーション・メニューから、「クエリー」フォルダーをクリックします。\n\n第 11 章 リレーショナル・レポート・スタイル  305\n\n\x0c2.「その他」   をクリックし、「生成された SQL/MDX を表示」を選択します。\n\n「生成された SQL」ウィンドウが開き、レポートのクエリーが表示されます。\n\n3.「クエリー結果」ウィンドウで、クエリー結果をクリックします。\n\nクエリーの SQL が表示されます。クエリーが複数のデータ・コンテナーで使用されている場合は、各デ\nータ・コンテナーに対してクエリー結果が生成されます。\n\n4.「生成された SQL/MDX」ウィンドウのドロップダウン・リストで、クエリーの実行時にデータベースに\n渡される SQL である「ネイティブ SQL」を選択することも、IBM Cognos Analytics - Reporting で使用\nされる汎用形式の SQL である「IBM Cognos SQL」を選択することもできます。Cognos SQL は、ネイ\nティブ SQL に変換されてからクエリーが実行されます。\n\n独自の SQL を使用したレポートの作成\n別のレポートなど、外部ソースの SQL を追加してレポートを作成できます。\n\nクエリーの SQL を編集する場合は、そのクエリーの「処理」プロパティーを「ローカルのみ」に変更する\n必要があります。\n\n手順', metadata={'source': 

In [19]:
query = "複数のプロンプトを複数にまとめて、フォームタイトルを付ける方法を教えてください"
result = qa({"query": query})
result

{'query': '複数のプロンプトを複数にまとめて、フォームタイトルを付ける方法を教えてください',
 'result': ' プロンプト・ページを作成し、レポートでの表示方法を指定することで、複数のプロンプトを複数にまとめて、フォームタイトルを付けることができます。ナビゲーション・メニューで「レポート」→「プロンプト・ページ」をクリックし、「ツールボックス」アイコンをクリックして「ページ」を「プロンプト・ページ」ボックスへドラッグし、作成したページをダブルクリックします。次に、「ツールボックス」アイコンをクリックし、「プロンプト」を拡張してから、プロンプト・ページへドラッグします。',
 'source_documents': [Document(page_content='プロンプト・テキストのカスタマイズ..............................................................................................374\nプロンプト値の指定........................................................................................................................... 375\nプロンプト・ボタンの追加................................................................................................................ 375\nカスケード・プロンプトの作成.........................................................................................................376\nツリー・プロンプトに表示されるデータのコントロール......................................................................377', metadata={'source': 'ug_cr_rptstd.pdf'}),
  Do

In [20]:
query = "プロンプトで複数の値を選択できるようにするにはどうすればよいですか"
result = qa({"query": query})
result

{'query': 'プロンプトで複数の値を選択できるようにするにはどうすればよいですか',
 'result': ' プロパティーウィンドウで「複数選択を有効にする」をオンにします。また、「自動送信」プロパティーを「はい」に設定する必要があります。',
 'source_documents': [Document(page_content='プロンプトの変更.................................................................................................................................... 372\nプロンプト・コントロール・インターフェースの変更.................................................................... 372\nユーザー入力を要求するプロンプトの指定...................................................................................... 373\nプロンプトでの複数の値の選択.........................................................................................................373\nプロンプトのステータスを表示または非表示にする方法................................................................ 374\nプロンプトのデフォルトの選択肢の指定.......................................................................................... 374', metadata={'source': 'ug_cr_rptstd.pdf'}),
  Document(page_content='をクリックして、レポート・ページをクリックします。\n3.「ページ」アイコン \n4. プロンプトに関連付けられているレポート・オブジェクトを